# General Analysis
Check in what particularly the 13 HTS reactions differ from the general reactions for those compounds
Use standard data science tools, e.g.. PCA, t-SNE etc. to investigate possible differences

## Libraries

In [53]:
import pandas as pd
import numpy as np
from matplotlib import pylab as plt
from matplotlib import cm
import seaborn as sns
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from matplotlib.lines import Line2D
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import umap

## Functions and definitions

In [54]:
def Add_Feature_Importance_PCA(score,coeff,labels=None):

        n = coeff.shape[0]

        for i in range(n):
            if abs(coeff[i,0]) < 0.2 and abs(coeff[i,1]) < 0.2:
                continue

            plt.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'black',alpha = 0.5)
            if labels is None:
                plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'grey', ha = 'center', va = 'center',fontsize=4)
            else:
                plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'grey', ha = 'center', va = 'center',fontsize=4)

## Main part

In [55]:
# Read solvent hazzardned file
##
solvent_hazzard = {}
fp = open('../data/processed/List_of_Solvents.tsv')
fp.readline()
for line in fp:
    tmp = line.strip().split('\t')
    solvent_hazzard[tmp[0]] = int(tmp[1])
print (solvent_hazzard)

{'methanol': 1, 'acetic acid': 2, 'chlorobenzene': 3, '1,4-dioxane': 5, 'N,N-dimethyl-formamide': 5, 'water': 1, 'nitromethane': 6, 'glycerol': 1, 'toluene': 3, 'acetonitrile': 3, 'ethyl acetate': 2, 'dichloromethane': 4, 'dimethyl sulfoxide': 3, 'ethanol': 1, 'chloroform': 6, '1,2-dichloro-ethane': 6, 'neat (no solvent)': 1}


In [56]:
# Read HCode severity list
####
hcode_severity = {}
fp =open('../data/processed/HCode_Severity.csv')
fp.readline()
for line in fp:
    tmp = line.strip().split(',')
    hcode_severity[tmp[0]] = float(tmp[1])
print (hcode_severity)

# Read Catalyst list
###
catalyst_hazzard_codes = {}
fp = open('../data/processed/Catalysts_Overview.tsv')
fp.readline()
for line in fp:
    tmp = line.strip().split('\t')
    #print (tmp)
    if tmp[1] not in catalyst_hazzard_codes:
        catalyst_hazzard_codes[tmp[1]] = {'GSH': [x for x in tmp[5].split(';') if x != ''],
                                    'H': [x for x in tmp[6].split(';') if x != ''],
                                    'P': [x for x in tmp[7].split(';') if x != '']}
    else:
        catalyst_hazzard_codes[tmp[1]]['GSH'].extend([x for x in tmp[5].split(';') if x != ''])
        catalyst_hazzard_codes[tmp[1]]['H'].extend([x for x in tmp[6].split(';') if x != ''])            
        catalyst_hazzard_codes[tmp[1]]['P'].extend([x for x in tmp[7].split(';') if x != ''])            
                         

# Calculate severity of a molecule
# Currently the calculation = SUM of the individual H_Codes
##
hazzard_values = []
catalyst_hazzard = {}
for key in catalyst_hazzard_codes:
    catalyst_hazzard[key] = 0.0
    H_codes = catalyst_hazzard_codes[key]['H']
    for h in H_codes:
        catalyst_hazzard[key] += hcode_severity[h]
    
    hazzard_values.append(catalyst_hazzard[key])

{'H200': 1.0, 'H201': 1.0, 'H202': 0.8333333333333334, 'H203': 0.6666666666666666, 'H204': 0.5, 'H205': 0.3333333333333333, 'H206': 1.0, 'H207': 0.5, 'H208': 0.25, 'H220': 1.0, 'H221': 0.75, 'H222': 1.0, 'H223': 0.75, 'H224': 1.0, 'H225': 0.75, 'H226': 0.5, 'H227': 0.25, 'H228': 0.75, 'H229': 0.5, 'H230': 0.5, 'H231': 0.5, 'H232': 1.0, 'H240': 1.0, 'H241': 0.8, 'H242': 0.4, 'H250': 1.0, 'H251': 1.0, 'H252': 0.75, 'H260': 1.0, 'H261': 0.5, 'H270': 1.0, 'H271': 1.0, 'H272': 0.5, 'H280': 0.5, 'H281': 0.5, 'H282': 1.0, 'H283': 0.75, 'H284': 0.5, 'H290': 1.0, 'H300': 1.0, 'H301': 0.5, 'H302': 0.25, 'H303': 0.1, 'H304': 1.0, 'H305': 0.75, 'H310': 1.0, 'H311': 0.5, 'H312': 0.25, 'H313': 0.1, 'H314': 1.0, 'H315': 0.75, 'H316': 0.5, 'H317': 1.0, 'H318': 1.0, 'H319': 0.75, 'H320': 0.5, 'H330': 1.0, 'H331': 0.5, 'H332': 0.25, 'H333': 0.1, 'H334': 1.0, 'H335': 0.5, 'H336': 0.5, 'H340': 1.0, 'H341': 0.75, 'H350': 1.0, 'H350i': 1.0, 'H351': 0.75, 'H360': 1.0, 'H360F': 1.0, 'H360D': 1.0, 'H360FD': 1.

### Load reactions

In [59]:
#Get compiled file of all analyzed reactions
reactions = pd.read_csv('../data/processed/All_Reactions.csv')
reactions.head()

#Total number of reactions (contains also reactions with missing values)
print (len(reactions))

595


#### Plot Temperature/Time/Yield

In [60]:
#reaxys parameters used for analysis (3 that could be used directly)
important_columns = ['Temperature (Reaction Details) [C]','Time (Reaction Details) [h]','Yield (numerical)']

#dictionary saving mean results
mean_results = {}

#Go through each parameter and create a boxplot/histogram (HTS vs. Literature); pooled for all compounds
for f in important_columns:
    print(f)
    
    #get HTS values
    real = [float(x) for x in reactions.loc[reactions['Type'] == 'Fabian'][f].values if x !='None']
    
    #Get literature values
    literature = [ float(x) for x in reactions.loc[reactions['Type'] == 'Other'][f].values if x !='None']
    
    #add mean to mean_dic
    mean_results[f] = np.mean(literature)
    
    plt.bar([0,1],[np.mean(real), np.mean(literature)],
            yerr = [1.95*np.std(real)/np.sqrt(len(real)),1.95*np.std(literature)/np.sqrt(len(literature))], color = ['#2291A5','grey'])
    
    plt.ylabel('Mean Result [95% CI]')
    plt.title('ZScore: %.2f' %((np.mean(real)-np.mean(literature))/np.std(literature)))
    plt.ylim([0,max([np.mean(real),np.mean(literature)+0.2*np.std(literature)])])
    plt.xticks([0,1],['Real','Litertature'])
    plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Box_'+f+'.pdf')
    plt.close()
    
    
    plt.title('ZScore: %.2f' %((np.mean(real)-np.mean(literature))/np.std(literature)))
    if len(list(set(real))) == 1:
        plt.boxplot(literature)
        plt.scatter([1],[real[0]],s=20, color='red',marker='x')
    else:
        plt.boxplot([real,literature])
        plt.xticks([1,2],['Real','Litertature'])
    plt.ylabel(f)
    
    plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Boxplot_'+f+'.pdf')
    plt.close()
    
    
    print('Number of values: %d' %len(literature))
    sns.distplot(literature,kde=False, rug=True)
    plt.axvline(real[0], lw=2, color='red')
    plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Histogram_'+f+'.pdf')
    plt.close()


Temperature (Reaction Details) [C]
Number of values: 479
Time (Reaction Details) [h]
Number of values: 538
Yield (numerical)
Number of values: 513


#### Plot: "Has catalyst"
same as previous/ just for having a Reagent/Catalyst (needs different handling)

In [22]:
f = 'Reagent/Catalyst'
real = [1 for x in reactions.loc[reactions['Type'] == 'Fabian'][f].values] 
literature = reactions.loc[reactions['Type'] == 'Other'][f].values

literature_result = []
for solvent in literature:
    
    if solvent != 'None':
        literature_result.append(1)
    else:
        literature_result.append(0)

literature = literature_result
       
mean_results[f] = np.mean(literature)    

plt.title('ZScore: %.2f' %((np.mean(real)-np.mean(literature))/np.std(literature)))
plt.bar([0,1],[np.mean(real), np.mean(literature)],
        yerr = [1.95*np.std(real)/np.sqrt(len(real)),1.95*np.std(literature)/np.sqrt(len(literature))], color = ['#2291A5','grey'])
plt.ylabel('Mean Result [95% CI]')
plt.ylim([0,max([np.mean(real),np.mean(literature)+0.2*np.std(literature)])])
plt.xticks([0,1],['Real','Litertature'])
plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Box_HasCatalyst.pdf')
plt.close()


plt.title('ZScore: %.2f' %((np.mean(real)-np.mean(literature))/np.std(literature)))
plt.boxplot(literature)
plt.scatter([1],[real[0]],s=20, color='red',marker='x')
plt.ylabel(f)
plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Boxplot_HasCatalyst.pdf')
plt.close()


print(len(literature))
sns.distplot(literature,kde=False, rug=True)
plt.axvline(real[0], lw=2, color='red')
plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Histogram_HasCatalyst.pdf')
plt.close()

581


#### Plot: "Solvent toxicity"
same as before just for solven toxicity (needs different handling)

In [23]:
f = 'Solvent (Reaction Details)'
real = [solvent_hazzard[x] for x in reactions.loc[reactions['Type'] == 'Fabian'][f].values] 
literature = reactions.loc[reactions['Type'] == 'Other'][f].values

literature_result = []
for solvent in literature:
    
    if solvent != 'None':
        tmp  = solvent.split(';')
        values = []
        for s in tmp:
            values.append(solvent_hazzard[s.strip()])
        literature_result.append(max(values))

literature = literature_result
       
mean_results[f] = np.mean(literature)    

plt.title('ZScore: %.2f' %((np.mean(real)-np.mean(literature))/np.std(literature)))
plt.bar([0,1],[np.mean(real), np.mean(literature)],
        yerr = [1.95*np.std(real)/np.sqrt(len(real)),1.95*np.std(literature)/np.sqrt(len(literature))], color = ['#2291A5','grey'])
plt.ylabel('Mean Result [95% CI]')
plt.ylim([0,max([np.mean(real),np.mean(literature)+0.2*np.std(literature)])])
plt.xticks([0,1],['Real','Litertature'])
plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Box_'+f+'.pdf')
plt.close()


plt.title('ZScore: %.2f' %((np.mean(real)-np.mean(literature))/np.std(literature)))
plt.boxplot(literature)
plt.scatter([1],[real[0]],s=20, color='red',marker='x')
plt.ylabel(f)
plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Boxplot_'+f+'.pdf')
plt.close()


print(len(literature))
sns.distplot(literature,kde=False, rug=True)
plt.axvline(real[0], lw=2, color='red')
plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Histogram_'+f+'.pdf')
plt.close()

452


In [24]:
# Make a pie chart for solvent hazzardness
##
data = []
for i in range(1,7):
    data.append(literature.count(i))

print (len(literature))
plt.title('Overivew hazardousness solvent [n = %d] ' %len(literature))

plt.pie(data, colors =['#1a9850','#91cf60','#d9ef8b','#fee08b','#fc8d59','#d73027'], labels=[str(x) for x in range(1,7)],  autopct='%1.1f%%', pctdistance=0.75)
plt.legend(['Recommended (R)','Between R/P','Problematic (P)', 'Between (P/H)', 'Hazardous (H)', 'Highly hazardous (HH)'], bbox_to_anchor=(1, 1),  prop={'size': 6}, frameon=False)
plt.savefig('../results/3_General_Analysis/Piechart_Solvents.pdf')
plt.close()

452


#### Plot: "Catalyst toxicity"
same as previous/ just for hazzardnes of Reagent/Catalyst (needs different handling)

In [52]:
f = 'Reagent/Catalyst'
real = [catalyst_hazzard[x] for x in reactions.loc[reactions['Type'] == 'Fabian'][f].values] 
literature = reactions.loc[reactions['Type'] == 'Other'][f].values

literature_result = []
for catalyst in literature:
    if catalyst != 'None' :

        literature_result.append(catalyst_hazzard[catalyst])

print (len(literature_result))  
literature = literature_result
       
mean_results[f] = np.mean(literature)    

plt.title('ZScore: %.2f' %((np.mean(real)-np.mean(literature))/np.std(literature)))
plt.bar([0,1],[np.mean(real), np.mean(literature)],
        yerr = [1.95*np.std(real)/np.sqrt(len(real)),1.95*np.std(literature)/np.sqrt(len(literature))], color = ['#2291A5','grey'])
plt.ylabel('Mean Result [95% CI]')
plt.ylim([0,max([np.mean(real),np.mean(literature)+0.2*np.std(literature)])])
plt.xticks([0,1],['Real','Litertature'])
plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Box_CatalystToxicity.pdf')
plt.close()


plt.title('ZScore: %.2f' %((np.mean(real)-np.mean(literature))/np.std(literature)))
plt.boxplot(literature)
plt.scatter([1],[real[0]],s=20, color='red',marker='x')
plt.ylabel(f)
plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Boxplot_CatalystToxicity.pdf')
plt.close()


print(len(literature))
sns.distplot(literature,kde=False, rug=True)
plt.axvline(real[0], lw=2, color='red')
plt.savefig('../results/3_General_Analysis/Feature_boxplots_and_histograms/Histogram_Box_CatalystToxicity.pdf')
plt.close()

484
484


In [28]:
# Make Pie chart for all catalyst/reagent hazzardness
##

data = []
for i in range(1,10):
    
    data.append(len([x for x in literature if x < i and x >= (i -1)]))

print (len(literature))
plt.title('Overivew hazardousness catalyst [n = %d] ' %len(literature))

plt.pie(data, colors =['#1a9850','#66bd63','#a6d96a','#d9ef8b','#ffffbf','#fee08b','#fdae61','#f46d43','#d73027'], labels=[str(x) for x in range(1,10)],  autopct='%1.1f%%', pctdistance=0.75)
plt.legend(['1','2','3','4','5','6','7','8','9'], bbox_to_anchor=(1, 1),  prop={'size': 6}, frameon=False)
plt.savefig('../results/3_General_Analysis/Piechart_Catalysts.pdf')
plt.close()

0.0
484


In [1]:
# Find max/min values (needed for POC normalization)
# Solvent and catalyst have trivial (=known) values
##
max_min = {'Solvent (Reaction Details)':{'max':6,'min':1}, 'Reagent/Catalyst':{'max':max(hazzard_values),'min':min(hazzard_values)}}

for f in important_columns:
    if f != 'Solvent (Reaction Details)' and f!= 'Reagent/Catalyst':
        tmp = reactions[f].copy()
        tmp.replace('None', np.nan, inplace=True)

        tmp = tmp.astype(float)
        maximum = (tmp.astype(float).max(skipna=True))
        minimum = (tmp.astype(float).min(skipna=True))

        max_min[f] = {'max':maximum,'min':minimum}

print (max_min)

NameError: name 'hazzard_values' is not defined

## Create Feature vectors

In [30]:
# This part creates a vector containing all values for the individual reactions
# Addtionally colors are associated, e.g., tab20 for the different compound number, RdYlGn for parameter values
########
colormap = cm.get_cmap('tab20',14)
colormap_gradient = cm.get_cmap('RdYlGn', 100)


#final data_vector containing all data (used for annotation)
all_Data_combined = []

# different color vectors (needed for the different plots later)
is_literature = []
colors_reactionNumber = []
colors_solvent = []
colors_catalysator = []
colors_yield = []
colors_has_catalysator = []

#used for colors (as some values are quite far off compared to other values, this is need to show a more linient grade of color)
manual_borders =  {'Temperature (Reaction Details) [C]': {'Max':100.0, 'Min':20.0},'Time (Reaction Details) [h]':{'Max':4.0, 'Min':0.0},'Yield (numerical)':{'Max':100.0, 'Min':70.0}}
colors_continues = {'Temperature (Reaction Details) [C]': [],'Time (Reaction Details) [h]':[],'Yield (numerical)':[]}

#contains the info for each reaction (to find them later)
labels = []
for row in reactions.iterrows():
    data = row[1]
    
    
    tmp_row = []
    
    #remove all reactions that are not fully annotated
    if  'None' in data[important_columns].values:
        continue
        
    #save label
    labels.append(data)    
    
    keep_row = True
    for f in important_columns:
        val = data[f]
        
        #this should never happen (see before; code used from before)
        if val != 'None':
            feature_val = float(val)
        else:
            feature_val = mean_results[f]
                    
        tmp_row.append(feature_val)
    
        #by far most reported yields are between 1 and 4.5
        normed = (feature_val - manual_borders[f]['Min'])/(manual_borders[f]['Max']-manual_borders[f]['Min'])
        normed = normed * 100
        if normed > 100:
            normed = 100
        
        #for the majority of features higher values are accociated with worse conditions (higher harzard, temperature etc.)
        if f != 'Yield (numerical)':
            normed = 100 - normed
        
        colors_continues[f].append(colormap_gradient(int(normed)))

        
    #SOLVENT
    #######
    solvent = data['Solvent (Reaction Details)']
    #print (solvent)
    if solvent != 'None':
        tmp  = solvent.split(';')
        values = []
        for s in tmp:
            values.append(solvent_hazzard[s.strip()])
        solvent_val = max(values)
    else:
        solvent_val = mean_results['Solvent (Reaction Details)']
    tmp_row.append(solvent_val)

    
    #by far most reported solvent are between 1 and 4.5
    normed = (solvent_val - 1)/(2-1)
    normed = normed * 100
    if normed > 100:
        normed = 100
    
    #invert for color (green good, red bad)
    normed = 100 - normed
    colors_solvent.append(colormap_gradient(int(normed)))
    
    
    
    #Catalystor toxicity
    #######
    catalyst = data['Reagent/Catalyst']
    #print (solvent)
    if catalyst != 'None':
        catalyst_val = catalyst_hazzard[catalyst]
    else:
        catalyst_val = mean_results['Reagent/Catalyst']
    tmp_row.append(catalyst_val)


    normed = (catalyst_val - 1)/(4-1)
    normed = normed * 100
    if normed > 100:
        normed = 100
    
    #invert for color (green good, red bad)
    normed = 100 - normed
    colors_catalysator.append(colormap_gradient(int(normed)))
    
    
    
    #Has Catalyst (good features?)
    ####
    
    if catalyst != 'None':
        colors_has_catalysator.append(colormap_gradient(0))
    else:
        colors_has_catalysator.append(colormap_gradient(100))
    
    
    # Combine vector
    ##
    all_Data_combined.append(tmp_row)
    
    #colors for reaction number
    #######
    colors_reactionNumber.append(colormap(data['Compound_ID']-1))

    #colors for reaction number
    #######
    if data['Yield (numerical)'] != 'None':
        val = float(data['Yield (numerical)'])
    else:
        val = float(mean_results['Yield (numerical)'])
    #by far most reported yields are between 70 and 100
    normed = (val - 70.0)/(100-70)
    normed = normed * 100
    if normed < 0:
        normed = 0
    colors_yield.append(colormap_gradient(int(normed)))

    
    if data['Type'] == 'Other':
        is_literature.append(True)
    else:
        is_literature.append(False)

In [31]:
# Create a final list of all reactions (including their references/source)
## 

#list of solvents with toxicity = 1
recommended_solvents  = []

#create output file
fp_out = open('../results/3_General_Analysis/Reaction_Table.tsv','w')
fp_out.write('Copound_ID\tIs_Literature\tIs_HTS\tTemperature[C]\tTime[h]\tYield[%]\tSolvent\tSolvent_Toxicity[Category]\tCatalyst\tCatalyst_Toxicity[Score]\tReaxysLink\tSource\n')

#go through all reactions
for l,d in zip(labels,all_Data_combined):
    
    #extract information
    solvent = l[3]
    catalyst = l[13]
    Compound_ID = l[11]
    reaction_type = l[12]
    article = l[10]
    link = l[9]
    
    #check for HTS or literature
    if reaction_type == 'Other':
        is_HTS = 'False'
        Is_Literature = 'True'
    else:
        is_HTS = 'True'
        Is_Literature = 'False'
        
    
    #Reactions that don't have a toxicity assumed got the mean toxicity value ca. 1.5 assigned
    if '.' in str(d[3]):
        solvent_toxicity = 'None'
    else:
        solvent_toxicity = str(d[3])
    
    fp_out.write(str(Compound_ID)+'\t'+Is_Literature+'\t'+is_HTS+'\t'+
                str(d[0])+'\t'+str(d[1])+'\t'+str(d[2])+'\t'+solvent+'\t'+
                 solvent_toxicity+'\t'+catalyst+'\t'+str(d[4])+'\t'+link+'\t'+article+'\n')
    
    #find recommended solvents
    if solvent_toxicity == '1' and is_HTS == 'False':
        recommended_solvents.append(solvent)
        
fp_out.close()

In [32]:
# Create a bar graph showing which recommded solvent appear most often
# This is important as direct comparison to HTS (which uses also water = R)
###
r_counts = Counter(recommended_solvents)
print(r_counts)

s_labels = []
s_count = []
s_percentages = []
for r in r_counts:
    s_labels.append(r)
    s_count.append(r_counts[r])
    s_percentages.append(r_counts[r]/len(recommended_solvents))

s_count, s_labels = zip(*sorted(zip(s_count, s_labels), reverse=True))

s_percentages, s_labels = zip(*sorted(zip(s_percentages, s_labels), reverse=True))

print(s_labels)
print([round(x,2)*100 for x in s_percentages])

plt.bar(range(0,len(s_count)),s_count, color='#2291A5')
plt.ylabel('Number of reactions')
plt.xticks(range(0,len(s_count)),s_labels, rotation=45, size=3)
plt.savefig("../results/3_General_Analysis/Category1_Solvents.pdf")
plt.close()
#plt.boar(5,5])

Counter({'ethanol': 108, 'water': 74, 'methanol': 34, 'neat (no solvent)': 31, 'ethanol; water': 19, 'ethanol; ethanol': 11, 'glycerol': 8, 'water; glycerol': 6, 'methanol; water': 2})
('neat (no solvent)', 'ethanol; water', 'ethanol', 'water', 'ethanol; ethanol', 'glycerol', 'water; glycerol', 'methanol; water', 'methanol')
[37.0, 25.0, 12.0, 11.0, 6.0, 4.0, 3.0, 2.0, 1.0]


In [33]:
#Rename feature names (reaxys feature names too long)
features = important_columns.copy()
features.append('Solvent Toxicity')
print ('Old Feature Names:')
print (features)
print ('--')
#features = ['Temperature','Time','Yield','Solvent Toxicity','Catalyst Toxicity', 'Has Catalyst']
features = ['Temperature','Time','Yield','Solvent Toxicity','Catalyst Toxicity']
print ('Old Feature Names:')
print (features)

Old Feature Names:
['Temperature (Reaction Details) [C]', 'Time (Reaction Details) [h]', 'Yield (numerical)', 'Solvent Toxicity']
--
Old Feature Names:
['Temperature', 'Time', 'Yield', 'Solvent Toxicity', 'Catalyst Toxicity']


### Check rank of Fabians reaction
See in which percentile HTS would score,e.g., being the hotest or cleanest

In [37]:
all_reactions = len(all_Data_combined)
print ("Number all reactions: %d" %all_reactions)

temperatue = all_Data_combined[117][0]
time = all_Data_combined[117][1]
percent_yield = all_Data_combined[117][2]
solvent_toxicity = all_Data_combined[117][3]
catalyst_toxicity = all_Data_combined[117][4]

temperature_rank = len([x for x in all_Data_combined if x[0] > temperatue])
print ('Number of reactions with higher temperature: %d' %temperature_rank)
print (temperature_rank/float(all_reactions))

time_rank = len([x for x in all_Data_combined if x[1] < time])
print ('Number of reactions with faster time: %d' %time_rank)
print (time_rank/float(all_reactions))

yield_rank = len([x for x in all_Data_combined if x[2] > percent_yield])
print ('Number of reactions with higher yield: %d' %yield_rank)
print (yield_rank/float(all_reactions))

solvent_rank = len([x for x in all_Data_combined if x[3] < solvent_toxicity])
print ('Number of reactions with less toxic solvent: %d' %solvent_rank)
print (solvent_rank/float(all_reactions))

catalyst_rank = len([x for x in all_Data_combined if x[4] < catalyst_toxicity])
print ('Number of reactions with less toxic catalyst: %d' %catalyst_rank)
print (catalyst_rank/float(all_reactions))

Number all reactions: 453
Number of reactions with higher temperature: 0
0.0
Number of reactions with faster time: 119
0.26269315673289184
Number of reactions with higher yield: 321
0.7086092715231788
Number of reactions with less toxic solvent: 0
0.0
Number of reactions with less toxic catalyst: 0
0.0


### Find 'to good to be true reactions'
Some reactions seem to achieve very good yields (>90) in low time, low temperature with almost no hazardness  
Check out if they have weird reagents

In [38]:
all_catalysts = []
perfect_catalysts  = []

#Output files containing reactions that report a almost too good to be true value of yield given the reaction parametrs
fp_out = open('../results/3_General_Analysis/Reagents_For_PeRfEcT_Reactions.csv','w')
fp_out.write('Reagent\n')
for r,l in zip(all_Data_combined,labels):
    all_catalysts.append(l['Reagent/Catalyst'])
    #Temperature < 25, time < 0.2h, Yield >90% and solven toxicity and catalyst toxicity <2
    if r[0] < 25 and r[1] < 0.2 and r[2] > 90 and r[3] < 2 and r[4] < 2:
        fp_out.write(l['Reagent/Catalyst']+'\n')
        perfect_catalysts.append(l['Reagent/Catalyst'])
fp_out.close()    
    

# Check how many of those reactions use a catalys
##
no_catalyst = perfect_catalysts.count('None')
has_catalyst = len(perfect_catalysts) - no_catalyst

plt.title('"Perfect" reactions having catalyst [n = %d] ' %len(perfect_catalysts))
plt.pie([has_catalyst,no_catalyst], colors =['#1a9850','#d73027'], labels=['Has catalyst', 'Has no catalyst'],  autopct='%1.1f%%', pctdistance=0.75)
plt.savefig('../results/3_General_Analysis/Piechart_PerfectR_Catalysts.pdf')
plt.close()



# Make pie chart of reaction of how often in general they use a catalyst
##
no_catalyst = all_catalysts.count('None')
has_catalyst = len(all_catalysts) - no_catalyst
plt.title('All reactions having catalyst [n = %d] ' %len(all_catalysts))
plt.pie([has_catalyst,no_catalyst], colors =['#1a9850','#d73027'], labels=['Has catalyst', 'Has no catalyst'],  autopct='%1.1f%%', pctdistance=0.75)
plt.savefig('../results/3_General_Analysis/Piechart_All_Catalysts.pdf')
plt.close()

### Make a PCA analysis

In [39]:
#create a figure legend
legend_elements = [Line2D([0], [0], marker='.', color='w', markerfacecolor='grey', label='Literature'),
                  Line2D([0], [0], marker='s', color='w', markerfacecolor='grey', label='Real')]
for i in list(set(reactions['Compound_ID'])):
    legend_elements.append(Line2D([0], [0], marker='o', color='w', markerfacecolor=colormap(i-1), label='Cmp '+str(i)))

In [40]:
#Scale values (use StandardScaler, i.e., all values scaled in standard deviations away from mean)
#Typically improves PCAs
x = StandardScaler().fit_transform(all_Data_combined)

#create PCA
pca = PCA(n_components=2)

#Fit PCA to scaled values
X = pca.fit_transform(x)

#check amount of variance explained by the indiviual principal components
components = pca.explained_variance_ratio_

print (features)
print(pca.components_ )

#use the previously defined colorsets
colorsets = {'Reaction':colors_reactionNumber,
             'Yield':colors_continues['Yield (numerical)'],'Time':colors_continues['Time (Reaction Details) [h]'],'Temperature':colors_continues['Temperature (Reaction Details) [C]'],
             "Solvent_Toxicity":colors_solvent, 'Catalysator_Toxcity':colors_catalysator, 'Has_Catalystor':colors_has_catalysator}

#go through all parameters, e.g., temperature, reaction time etc.
for c_set in colorsets:
    print (c_set)
    for row,literat, reaction_id in zip(X,is_literature,colorsets[c_set]):
        if literat == True:
            plt.scatter(row[0],row[1],color=reaction_id, marker='.', alpha=0.8, lw=0 , s=50)
        else:
            plt.scatter(row[0],row[1],color=reaction_id, marker='s', alpha = 0.8, lw=0, s=50)

    #add arrows showing the importance of specific features to the principal components
    Add_Feature_Importance_PCA(X[:,0:2],np.transpose(pca.components_[0:2, :]),features)
    
    plt.title(c_set)
    
    if c_set == 'Reaction':
        plt.legend(handles=legend_elements,loc='best',  prop={'size': 6}, frameon=False)
    plt.xlabel('PC1 [%.2f]' %components[0])
    plt.ylabel('PC2 [%.2f]' %components[1])
    #plt.show()
    plt.savefig('../results/3_General_Analysis/PCA/PCA_withFeatures_'+c_set+'.pdf')
    plt.close()

['Temperature', 'Time', 'Yield', 'Solvent Toxicity', 'Catalyst Toxicity']
[[ 0.23419981  0.57752123 -0.52377685  0.36236055 -0.45384174]
 [-0.75973898  0.21511451 -0.02695247  0.51861081  0.32686207]]
Reaction
Yield
Time
Temperature
Solvent_Toxicity
Catalysator_Toxcity
Has_Catalystor


### TSNE

Make a TSNE analysis (show feature expression as well as a cluster analysis)

In [41]:
#create a figure legend
legend_elements = [Line2D([0], [0], marker='.', color='w', markerfacecolor='grey', label='Literature'),
                  Line2D([0], [0], marker='s', color='w', markerfacecolor='grey', label='Real')]
for i in list(set(reactions['Compound_ID'])):
    legend_elements.append(Line2D([0], [0], marker='o', color='w', markerfacecolor=colormap(i-1), label='Cmp '+str(i)))

In [42]:
#define random state (to get same results)
rand_state = 400

#Initialize T-SNE
X_TSNE = TSNE(n_components=2,perplexity=45.0, random_state=rand_state).fit_transform(x)

#Same as for PCA (overaly colorset for each parameter), then plot T-SNE
for c_set in colorsets:
    print (c_set)
    for row,literat, reaction_id in zip(X_TSNE,is_literature,colorsets[c_set]):
        if literat == True:
            plt.scatter(row[0],row[1],color=reaction_id, marker='.', alpha=0.8)
        else:
            plt.scatter(row[0],row[1],color=reaction_id, marker='s', alpha = 0.8)

    plt.title(c_set)
    if c_set == 'Reaction':
        plt.legend(handles=legend_elements,loc='best',  prop={'size': 6}, frameon=False)
    plt.xlabel('tsne1')
    plt.ylabel('tsne2')
    plt.savefig('../results/3_General_Analysis/TSNE/TSNE_'+c_set+'.pdf')
    plt.close()

Reaction
Yield
Time
Temperature
Solvent_Toxicity
Catalysator_Toxcity
Has_Catalystor


In [43]:
# Make K-Means clustering, create elbow curve (showing optimal amount of k)
# Analyse between 1 and 10
###
number_to_analyse = 10

#vector containing remaining distortion
distorsions = []

#make K-mean clustering between k=1 and k=9
for k in range(1, number_to_analyse):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X_TSNE)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(1, number_to_analyse), distorsions)
plt.title('Elbow curve')
plt.ylabel('Sum Squared Distances to closest cluster center')
plt.xlabel('Number of clusters')
plt.savefig('../results/3_General_Analysis/TSNE/TSNE_ElbowCurve.pdf')
plt.close()

In [44]:
# use k = 8, according to elbow curve
##
num_clusters = 8

#define new colormap (for different cluster)
colormap = cm.get_cmap('tab20')

#make K-Mean clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=rand_state,max_iter=2000).fit(X_TSNE)

#Plot the scatterplot, make different marker for HTS (square) and literature (point)
for row,literat, cluster in zip(X_TSNE,is_literature,kmeans.labels_):
    if literat == True:
        plt.scatter(row[0],row[1],color=colormap(cluster), marker='.')
    else:
        plt.scatter(row[0],row[1],color=colormap(cluster), marker='s')

        #make figure legend (HTS vs. literature)
        legend_elements = [Line2D([0], [0], marker='.', color='w', markerfacecolor='grey', label='Literature'),
                  Line2D([0], [0], marker='s', color='w', markerfacecolor='grey', label='Real')]

#add cluster color to figure legend
for i in range(0,num_clusters):
    legend_elements.append(Line2D([0], [0], marker='o', color='w', markerfacecolor=colormap(i), label='Cluster '+str(i)))

plt.legend(handles=legend_elements,loc='best',  prop={'size': 6}, frameon=False)  
plt.xlabel('tsne1')
plt.ylabel('tsne2')
plt.savefig('../results/3_General_Analysis/TSNE/TSNE_withCluster.pdf')
plt.close()

In [45]:
#TSNE in gray
for row,literat, cluster in zip(X_TSNE,is_literature,kmeans.labels_):
    if literat == True:
        plt.scatter(row[0],row[1],color='grey', marker='.', alpha = 0.4, linewidth=0)
    else:
        plt.scatter(row[0],row[1],color='grey', marker='s', alpha = 0.4, linewidth=0)


        legend_elements = [Line2D([0], [0], marker='.', color='w', markerfacecolor='grey', label='Literature'),
                  Line2D([0], [0], marker='s', color='w', markerfacecolor='grey', label='Real')]
for i in range(0,num_clusters):
    legend_elements.append(Line2D([0], [0], marker='o', color='w', markerfacecolor=colormap(i), label='Cluster '+str(i)))

plt.legend(handles=legend_elements,loc='best',  prop={'size': 6}, frameon=False)
        
plt.xlabel('tsne1')
plt.ylabel('tsne2')
#plt.show()
plt.savefig('../results/3_General_Analysis/TSNE/TSNE_grey.pdf')
plt.close()

In [46]:
# Make an analysis showing the feature distribution for the individual clusters, see if certain cluster correspond to certain parameter expression
#
###

#initiliaze dictionary (contain final cluster results)
per_cluster_results = {}
for i in range(0,num_clusters):
    per_cluster_results[i] = []

#add data to cluster dic
for data,cluster in zip(x,kmeans.labels_):
        per_cluster_results[cluster].append(data)

# Make a boxplot and a bar plot for EACH cluster
##
for i in range(0,num_clusters):      
    tmp = []
    for f in range(0,len(features)):
        tmp.append(np.array(per_cluster_results[i])[:,f])
    
    # Make boxplot
    ##
    bplot = plt.boxplot(tmp,patch_artist=True)
    # fill with colors
    colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e']
    for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)
    
    plt.xticks(range(1,len(features)+1),features, rotation=14)
    plt.axhline(0, color='grey', ls='--')
    plt.savefig('../results/3_General_Analysis/TSNE/TSNE_CLUSTER_Results/Cluster_'+str(i)+'_boxplot.pdf')
    plt.close()
    

    # Make bar plot
    ##
    plt.bar(range(0,len(features)),[np.mean(d) for d in tmp], yerr = [np.std(d) for d in tmp], color = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e'])
    plt.xticks(range(0,len(features)+1),features)
    plt.axhline(0, color='grey', ls='--')
    plt.ylim([-3.5,4])
    plt.savefig('../results/3_General_Analysis/TSNE/TSNE_CLUSTER_Results/Cluster_'+str(i)+'_barplot.pdf')
    plt.close()

### UMAP
Same as for T-SNE using UMAP (not used in final paper)

In [47]:
#create a figure legend
legend_elements = [Line2D([0], [0], marker='.', color='w', markerfacecolor='grey', label='Literature'),
                  Line2D([0], [0], marker='s', color='w', markerfacecolor='grey', label='Real')]
for i in list(set(reactions['Compound_ID'])):
    legend_elements.append(Line2D([0], [0], marker='o', color='w', markerfacecolor=colormap(i-1), label='Cmp '+str(i)))

In [48]:
# Initialize UMAP and fit data
reducer = umap.UMAP(n_neighbors=25, min_dist=0.6, metric='correlation', random_state=2500)
X_UMAP = reducer.fit_transform(x)

#Same as for PCA (overaly colorset for each parameter), then plot T-SNE
for c_set in colorsets:
    print (c_set)
    for row,literat, reaction_id in zip(X_UMAP,is_literature,colorsets[c_set]):
        if literat == True:
            plt.scatter(row[0],row[1],color=reaction_id, marker='.', alpha=1)
        else:
            plt.scatter(row[0],row[1],color=reaction_id, marker='s', alpha = 1)

    plt.title(c_set)
    if c_set == 'Reaction':
        plt.legend(handles=legend_elements,loc='best',  prop={'size': 6}, frameon=False)
    plt.xlabel('UMAP1')
    plt.ylabel('UMAP2')
    #plt.show()
    plt.savefig('../results/3_General_Analysis/UMAP/UMAP_'+c_set+'.pdf')
    plt.close()


Reaction
Yield
Time
Temperature
Solvent_Toxicity
Catalysator_Toxcity
Has_Catalystor


In [49]:
#Make elbow curve to identify ideal amount of clusters
num_clusters = 10

distorsions = []
for k in range(1, num_clusters):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X_UMAP)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(1, num_clusters), distorsions)
plt.title('Elbow curve')
plt.ylabel('Sum Squared Distances to closest cluster center')
plt.xlabel('Number of clusters')
plt.savefig('../results/3_General_Analysis/UMAP/UMAP_ElbowCurve.pdf')
plt.close()

In [50]:
#Use same number of clusters as identified for T-SNE = 8
num_clusters = 8

#Make colormap for different clusters
colormap = cm.get_cmap('tab20')

#Make K-Mean clustering for k=8
kmeans = KMeans(n_clusters=num_clusters, random_state=rand_state,max_iter=2000).fit(X_UMAP)

#Plot the scatterplot, make different marker for HTS (square) and literature (point)
for row,literat, cluster in zip(X_UMAP,is_literature,kmeans.labels_):
    if literat == True:
        plt.scatter(row[0],row[1],color=colormap(cluster), marker='.')
    else:
        plt.scatter(row[0],row[1],color=colormap(cluster), marker='s')

        #make figure legend (HTS vs. literature)
        legend_elements = [Line2D([0], [0], marker='.', color='w', markerfacecolor='grey', label='Literature'),
                  Line2D([0], [0], marker='s', color='w', markerfacecolor='grey', label='Real')]
#add cluster color to figure legend
for i in range(0,num_clusters):
    legend_elements.append(Line2D([0], [0], marker='o', color='w', markerfacecolor=colormap(i), label='Cluster '+str(i)))

plt.legend(handles=legend_elements,loc='best',  prop={'size': 6}, frameon=False)
        
plt.xlabel('umap1')
plt.ylabel('umap2')
plt.savefig('../results/3_General_Analysis/UMAP/UMAP_withCluster.pdf')
plt.close()

In [51]:
# Make an analysis showing the feature distribution for the individual clusters, see if certain cluster correspond to certain parameter expression
#
###

#initiliaze dictionary (contain final cluster results)
per_cluster_results = {}
for i in range(0,num_clusters):
    per_cluster_results[i] = []

#add data to cluster dic
for data,cluster in zip(x,kmeans.labels_):
        per_cluster_results[cluster].append(data)

# Make a boxplot and a bar plot for EACH cluster
##        
for i in range(0,num_clusters):      
    tmp = []
    for f in range(0,len(features)):
        tmp.append(np.array(per_cluster_results[i])[:,f])
    
    # Make boxplot
    ##
    plt.boxplot(tmp)
    plt.xticks(range(1,len(features)+1),features)
    plt.axhline(0, color='grey', ls='--')
    plt.savefig('../results/3_General_Analysis/UMAP/UMAP_CLUSTER_Results/Cluster_'+str(i)+'_boxplot.pdf')
    plt.close()
    
    # Make barplot
    ##
    plt.bar(range(0,len(features)),[np.mean(d) for d in tmp], yerr = [np.std(d) for d in tmp], color = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e'])
    plt.xticks(range(0,len(features)+1),features)
    plt.axhline(0, color='grey', ls='--')
    #plt.ylim([-5,5])
    plt.savefig('../results/3_General_Analysis/UMAP/UMAP_CLUSTER_Results/Cluster_'+str(i)+'_barplot.pdf')
    plt.close()